We are ready for Hamiltonian Monte Carlo methods. We need to specifically understand the NUTs sampler.

In [5]:
import numpy as np

In [6]:
# Define the target distribution (e.g., a standard Gaussian)
def target_log_prob(x):
    return -0.5 * np.sum(x**2)

# Gradient of the target log-probability
def grad_log_prob(x):
    return -x

# Leapfrog integrator
def leapfrog(x, p, step_size, num_steps):
    x_new = np.copy(x)
    p_new = np.copy(p)
    
    # Half step for momentum at the beginning
    p_new -= 0.5 * step_size * grad_log_prob(x_new)
    
    # Full step for position
    for _ in range(num_steps):
        x_new += step_size * p_new
        # Full step for momentum (except last iteration)
        if _ != num_steps - 1:
            p_new -= step_size * grad_log_prob(x_new)
    
    # Half step for momentum at the end
    p_new -= 0.5 * step_size * grad_log_prob(x_new)
    
    return x_new, p_new

# Hamiltonian function (H = Kinetic + Potential)
def hamiltonian(x, p):
    kinetic = 0.5 * np.sum(p**2)  # Kinetic energy from momentum (Gaussian)
    potential = -target_log_prob(x)  # Potential energy from target distribution
    return kinetic + potential

# HMC sampler
def hmc_sampler(init_x, step_size, num_steps, num_samples):
    samples = []
    x = np.copy(init_x)
    
    for _ in range(num_samples):
        # Sample momentum from standard normal
        p = np.random.randn(*x.shape)
        
        # Save the current state
        current_x = np.copy(x)
        current_p = np.copy(p)
        
        # Perform leapfrog integration
        new_x, new_p = leapfrog(x, p, step_size, num_steps)
        
        # Calculate Hamiltonians
        current_H = hamiltonian(current_x, current_p)
        new_H = hamiltonian(new_x, new_p)
        
        # Metropolis-Hastings correction
        if np.random.rand() < np.exp(current_H - new_H):
            x = new_x  # Accept the new state
        # else: reject and keep the current state
        
        samples.append(np.copy(x))
    
    return np.array(samples)



In [7]:
# Parameters
init_x = np.array([0.0])  # Initial position
step_size = 0.1  # Step size for leapfrog
num_steps = 10  # Number of leapfrog steps
num_samples = 1000  # Number of samples to generate

# Run HMC
samples = hmc_sampler(init_x, step_size, num_steps, num_samples)

# Print the first few samples
print(samples[:10])

[[ 0.        ]
 [ 0.        ]
 [-0.62109831]
 [-0.18447019]
 [-0.64700874]
 [-0.64700874]
 [-0.64700874]
 [ 0.84892764]
 [ 1.1016588 ]
 [ 0.87966707]]
